In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df = df[['Neighborhood','YrSold','SalePrice']]
df = df.dropna()
train_raw = df[df.YrSold < 2010]
test_raw = df[df.YrSold >= 2010]

In [5]:
train_raw.Neighborhood.value_counts()

NAmes      192
CollgCr    137
OldTown     99
Edwards     89
Somerst     78
Gilbert     73
NridgHt     69
NWAmes      66
Sawyer      58
BrkSide     56
Crawfor     48
SawyerW     46
Mitchel     45
NoRidge     34
Timber      33
IDOTRR      31
ClearCr     24
SWISU       22
StoneBr     20
Blmngtn     16
BrDale      15
MeadowV     14
Veenker     11
NPkVill      7
Blueste      2
Name: Neighborhood, dtype: int64

# OneHotEncoder

<font color='red'>Assignment 2.1:</font> **Neighborhood** is a categorical feature. It has to be transformed before feeding it to a machine learning model. Please transform the column in train_raw to a data frame X_train using **OneHotEncoder**. 

In [6]:
cat = df.Neighborhood.unique().tolist()

In [7]:
from sklearn.preprocessing import OneHotEncoder
X_train = OneHotEncoder(categories = [cat],handle_unknown='ignore').fit(train_raw[['Neighborhood']])

In [8]:
X_train = X_train.transform(train_raw[['Neighborhood']]).toarray()

In [9]:
X_train.shape

(1285, 25)

In [10]:
from sklearn.preprocessing import OneHotEncoder

X_train = OneHotEncoder(categories = [cat],handle_unknown='ignore', sparse = True).fit(train_raw[['Neighborhood']])
X_train = X_train.transform(train_raw[['Neighborhood']]).toarray()

In [11]:
X_train.shape

(1285, 25)

<font color='red'>Question 2.2:</font> What does OneHotEncoder do? Why do we set handle_unknown='ignore'? What is a sparse matrix (try set sparse=True and repeat to see what a sparse matrix looks like).

One-hot encoding can be used to transform one or more categorical features into numerical dummy features useful for training machine learning model. One-hot encoding is also called dummy encoding due to the fact that the transformation of categorical features results into dummy features.

When the model sees some new feature that it did not come across during the fitting process it throws an error but with handle_unknown='ignore' the encoder will ignore the unknown categories and no error will be raised.

In Machine learning model, we get sparse matrix when we have lots of zeroes in the resulting array. When we have lots of non zero entries the memory usage will be more.


<font color='red'>Assignment 2.3:</font> Built a linear regression model and use X_train to train the model, then use it to predict **SalePrice**. What is the R2 score of this model? Hint: you need to create y_train, X_test, y_test first.

In [12]:
features = list(train_raw.columns)
target = "SalePrice"
features.remove(target)

y_train = train_raw[target].copy()

In [13]:


features_test = list(test_raw.columns)

#assigning target variable name
target = "SalePrice"
features_test.remove(target)

X_test = test_raw[features_test].copy()
y_test = test_raw[target].copy()

In [14]:
import numpy as np

X_train = np.hstack((train_raw[['YrSold']].values,X_train))

In [15]:
from sklearn.preprocessing import OneHotEncoder

X_test = OneHotEncoder(categories = [cat],handle_unknown='ignore', sparse = True).fit(test_raw[['Neighborhood']])
X_test = X_test.transform(test_raw[['Neighborhood']]).toarray()

In [16]:
X_test = np.hstack((test_raw[['YrSold']].values,X_test))

In [17]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [18]:
y_train_pred = model.predict(X_train)

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

r2 = r2_score(y_train,y_train_pred)
print('R^2: {:.2f}'.format(r2))

R^2: 0.55


In [20]:
y_test_pred = model.predict(X_test)

In [21]:
r2 = r2_score(y_test,y_test_pred)
print('R2:',r2)

R2: 0.5374564394624182


# Combining numerical features and categorical features

In [22]:
df = pd.read_csv('train.csv')

num_features = ['LotFrontage','LotArea','BsmtFinSF1',
        'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF','1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea','TotRmsAbvGrd',
        'GarageArea','OpenPorchSF']
cat_features = ['Neighborhood','BldgType']

df = df[num_features+cat_features+['YrSold','SalePrice']]
df = df.dropna()
train_raw = df[df.YrSold < 2010]
test_raw = df[df.YrSold >= 2010]

<font color='red'>Assignment 2.4:</font> Now it is time to combine both numerical features and categorical features into our model! 

Create a **ColumnTransformer** to transform above num_features and cat_features at the same time. Specifically, apply StandardScaler to num_features and OneHotEncoder to cat_features. Then wrap the ColumnTransformer with LinearRegression to predict **SalepPrice**. 

How does the R2 score look like now?

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

scaler = StandardScaler()
enc = OneHotEncoder(handle_unknown='ignore')

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import  make_pipeline

categorical = df.dtypes == object

preprocess = make_column_transformer(
 (StandardScaler(), num_features),
 (OneHotEncoder(handle_unknown='ignore'), cat_features))

model = make_pipeline(preprocess, LinearRegression())

In [25]:
features = list(train_raw.columns)
target = "SalePrice"
features.remove(target)

X_train = train_raw[features].copy()
y_train = train_raw[target].copy()

In [26]:
#Answer
#Extracting column names
features_test = list(test_raw.columns)

#assigning target variable name
target = "SalePrice"
features_test.remove(target)

X_test = test_raw[features_test].copy()
y_test = test_raw[target].copy()

In [27]:
X_train.shape

(1058, 16)

In [28]:
y_train.shape

(1058,)

In [29]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['LotFrontage', 'LotArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'TotRmsAbvGrd', 'GarageArea',
                                                   'OpenPorchSF']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Neighborhood',
                                                   'BldgType'])

In [30]:
y_train_pred = model.predict(X_train)

In [31]:
#training model
r2_score(y_train,y_train_pred)

0.799293334278395

In [32]:
y_test_pred = model.predict(X_test)

In [33]:
#testing model
r2_score(y_test,y_test_pred)

0.8365077929392188

R2 value has improved

# SimpleImputer

In [79]:
df = pd.read_csv('train.csv')

num_features = ['LotFrontage','LotArea','BsmtFinSF1',
        'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF','1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea','TotRmsAbvGrd',
        'GarageArea','OpenPorchSF']
cat_features = ['Neighborhood','BldgType']

train_raw = df[df.YrSold < 2010]
test_raw = df[df.YrSold >= 2010]

<font color='red'>Assignment 2.5:</font> In above, your ColumnTransformer consists of a scaler for numerical columns and a one hot encoder for categorical columns. However, the ColumnTransformer can also consist of pipelines. 

Create a pipleline for numerical columns using **SimpleImputer(strategy='median')** followed by the scaler, and a pipeline for categorical columns using **SimpleImputer(strategy='most_frequent', missing_values='nan')** followed by OneHotEncoder, then use these two pipelines to create the ColumnTransformer, then train the linear regression model.

What are the pros and cons of using this approach compared with the previous approach?

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [50]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values='nan')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [51]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

In [52]:
features = list(train_raw.columns)
target = "SalePrice"
features.remove(target)

X_train = train_raw[features].copy()
y_train = train_raw[target].copy()

In [53]:
#Extracting column names
features_test = list(test_raw.columns)

#assigning target variable name
target = "SalePrice"
features_test.remove(target)

X_test = test_raw[features_test].copy()
y_test = test_raw[target].copy()

In [54]:
model = make_pipeline(preprocessor, LinearRegression())
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['LotFrontage', 'LotArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'TotRmsAbvGrd', 'GarageArea',
                                                   'OpenPorchSF']),
                     

In [55]:
y_train_pred = model.predict(X_train)

In [56]:
r2_score(y_train,y_train_pred)

0.799293334278395

In [57]:
#Predicting for Test Dataset
y_test_pred = model.predict(X_test)

In [58]:
r2_score(y_test,y_test_pred)

0.8365077929392188